In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Check out the part1: https://www.kaggle.com/code/gamirandads/introduction-to-pyspark-part1

#### If the notebook was usefull in any way, please vote up!

### Function in the notebook

In this notebook we will work with some types of **join** available in PySpark.

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_Introdution_part2')
    .getOrCreate() #If there were another prev session, It would start it again
)

In [ ]:
customers_d = spark.read.csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv',
                                   header=True, 
                                   inferSchema=True, #Will infer the variable types
                                   sep = ",")

orders_d = spark.read.csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv',
                                   header=True, 
                                   inferSchema=True, #Will infer the variable types
                                   sep = ",")

reviews_d = spark.read.csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv',
                                   header=True, 
                                   inferSchema=True, #Will infer the variable types
                                   sep = ",")

payments_d = spark.read.csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv',
                                   header=True, 
                                   inferSchema=True, #Will infer the variable types
                                   sep = ",")

In [ ]:
customers_d.show(5)

print((customers_d.count(), len(customers_d.columns)))

In [ ]:
orders_d.show(5)

print((orders_d.count(), len(orders_d.columns)))

In [ ]:
reviews_d.show(5)

print((reviews_d.count(), len(reviews_d.columns)))

In [ ]:
payments_d.show(5)

print((payments_d.count(), len(payments_d.columns)))

You can notice that customers_d and orders_d have a commom variable, customer_id, as orders_d and reviews_d has order_id. We can connect both datasets through the join function. PySpark has a few options of join available:

- Inner Join: Returns only the rows with matching keys in both DataFrames.
- Left Join: Returns all rows from the left DataFrame and matching rows from the right DataFrame.
- Right Join: Returns all rows from the right DataFrame and matching rows from the left DataFrame.
- Full Outer Join: Returns all rows from both DataFrames, including matching and non-matching rows.
- Left Anti Join: Returns all rows from the left DataFrame where there is no match in the right DataFrame.
- Left Semi Join: Returns all rows from the left DataFrame where there is a match in the right DataFrame.

### Inner join

For this example we want in the same dataset the orderes and their reviews.

In [ ]:
inner_example = orders_d.join(reviews_d, on = 'order_id', how = 'inner')

inner_example.show(n = 3, vertical=True)

print((inner_example.count(), len(inner_example.columns)))

You can notice that in the inner_example we have less observations than orders_d. Maybe some orders don't have reviews yet, so in the inner_example we "selected" only orders with reviews.

### Left join

In this example we want in the same dataset the customer "main" info and their orders.

In [ ]:
left_example = customers_d.join(orders_d, on = 'customer_id', how = 'left') #or leftouter, left_outer

left_example.show(n = 3, vertical=True)

print((left_example.count(), len(left_example.columns)))

In [ ]:
for coluna in left_example.columns:
    print(coluna, left_example.filter(left_example[coluna].isNull()).count())

Note we have the same number of observations in both customers_d and left_example, as expected since we are using left join.

In [ ]:
for coluna in orders_d.columns:
    print(coluna, orders_d.filter(orders_d[coluna].isNull()).count())

Also we have the same null seen in orders, in left_example.

### Right join

In [ ]:
right_example = customers_d.join(orders_d, on = 'customer_id', how = 'rightouter') #or right, right_outer

right_example.show(n = 3, vertical=True)

print((right_example.count(), len(right_example.columns)))

As costumers_d and orders_d have the same number of observations (the same observations), right and left join returns us the same result.

### Full join

In [ ]:
full_example = orders_d.join(reviews_d, on = 'order_id', how = 'full') #or outer, fullouter

full_example.show(n = 3, vertical=True)

print((full_example.count(), len(full_example.columns)))

### Left Anti Join

Remember when we create inner_example, we had less observations than in orders_d, which could be orders with no review. One way to see those orders is using left anti join

In [ ]:
left_anti_example = orders_d.join(reviews_d, on = 'order_id', how = 'leftanti') # or anti, left_anti

left_anti_example.show(n = 3, vertical=True)

print((left_anti_example.count(), len(left_anti_example.columns)))

So, there is 768 orders with no review.

### Left Semi Join

Left semi join is very similar to inner join. The only difference is that inner join returns the columns from both datasets and left semi join returns only the columns from the left dataset.

In [ ]:
left_semi_example = orders_d.join(reviews_d, on = 'order_id', how = 'leftsemi') # or semi, left_semi

left_semi_example.show(n = 3, vertical=True)

print((left_semi_example.count(), len(left_semi_example.columns)))

Source: https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/#pyspark-left-outer-join